In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import torchvision
import torchvision.datasets
import torchvision.transforms as transforms

import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-white')



import random
import math
import os

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
class config:
    seed = 42
    device = "cuda:0"    
        
    lr = 1e-3
    epochs = 25
    batch_size = 32
    num_workers = 4
    train_5_folds = True

def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  # type: ignore

seed_everything(config.seed)

In [4]:
use_cuda = torch.cuda.is_available()

device = torch.device("cuda" if use_cuda else "cpu")

device

device(type='cuda')

In [5]:
transform_train = transforms.Compose([transforms.ToTensor(),
                                     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616)),
                                      transforms.RandomHorizontalFlip(p=0.5),
                                      transforms.RandomVerticalFlip(p=0.5),
                                      transforms.RandomGrayscale(p=1)])


transform  = transforms.Compose([
        transforms.ToTensor(),
         transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))])


In [6]:
trainse11 = torchvision.datasets.CIFAR10(root='/content/drive/MyDrive/share/cafir_10',
                                        train=True,
                                        download=True,
                                        transform=transform_train)

vaildset = torchvision.datasets.CIFAR10(root='/content/drive/MyDrive/share/cafir_10',
                                        train=False,
                                        download=True,
                                        transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [7]:
trainset = trainse11 

testset = torchvision.datasets.ImageFolder(root = "/content/drive/MyDrive/share/images/Statistical_Deep_Image",
                                           transform = transform)

In [8]:
train_loader = DataLoader(trainset,
                          batch_size = 16,
                          shuffle=True,
                          num_workers=2)

vaild_loader = DataLoader(vaildset,
                          batch_size=32,
                          shuffle=False,
                          num_workers=2)


test_loader = DataLoader(testset,
                          batch_size=32,
                          shuffle=False,
                          num_workers=2)

모델 넣기 

In [9]:
!pip install timm
import timm




Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 548 kB 33.0 MB/s 
     |████████████████████████████████| 182 kB 54.6 MB/s 


In [10]:
net = timm.create_model('vgg13', pretrained=False, num_classes=10).to(device)

criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

total=0 
correct = 0

In [14]:
optimizer = optim.SGD(net.parameters(), lr=0.00001, momentum=0.9)

In [15]:
for epoch in range(50):
  running_loss = 0.0

  for i, data in enumerate(train_loader, 0):
    inputs, labels = data[0].to(device), data[1].to(device)

    optimizer.zero_grad()
    outputs= net(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
    if i % 750 ==749:
      print("Epoch: {},Batch : {}, Loss:{}".format(epoch+1, i+1, running_loss/2000))
      running_loss = 0.0

  with torch.no_grad():
    val_loss = 0.0
    for k, data1 in enumerate(vaild_loader, 0):
      val_inputs, val_label = data1[0].to(device), data1[1].to(device)
      val_output = net(val_inputs)
      v_loss = criterion(val_output, val_label)
      val_loss += v_loss
      _, predicted = torch.max(val_output.data,1)
      total += val_label.size(0)
      correct += (predicted == val_label).sum().item()
  print("validation loss {}".format(val_loss))
  print("vaildset Accuracy  : {}".format(100* correct/total))
  total=0
  correct=0
  with torch.no_grad():
    for data in test_loader:
      images, labels = data[0].to(device), data[1].to(device)
      outputs= net(images)
      _, predicted = torch.max(outputs.data,1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()

  print("testset Accuracy  : {}".format(100* correct/total))
  total=0
  correct=0

Epoch: 1,Batch : 750, Loss:0.042605665745853914
Epoch: 1,Batch : 1500, Loss:0.03824595493226662
Epoch: 1,Batch : 2250, Loss:0.03932466235419815
Epoch: 1,Batch : 3000, Loss:0.03988986485099304
validation loss 242.55908203125
vaildset Accuracy  : 80.16
testset Accuracy  : 62.65
Epoch: 2,Batch : 750, Loss:0.039803453710104804
Epoch: 2,Batch : 1500, Loss:0.03491679462770116
Epoch: 2,Batch : 2250, Loss:0.040205888191761914
Epoch: 2,Batch : 3000, Loss:0.03574468723853352
validation loss 244.12442016601562
vaildset Accuracy  : 80.08
testset Accuracy  : 62.75
Epoch: 3,Batch : 750, Loss:0.03459956330452405
Epoch: 3,Batch : 1500, Loss:0.03802076849181321
Epoch: 3,Batch : 2250, Loss:0.03712391294570989
Epoch: 3,Batch : 3000, Loss:0.03710891104885377
validation loss 244.703125
vaildset Accuracy  : 80.14
testset Accuracy  : 62.95
Epoch: 4,Batch : 750, Loss:0.035836054245271955
Epoch: 4,Batch : 1500, Loss:0.037938876941218044
Epoch: 4,Batch : 2250, Loss:0.03475302923252457
Epoch: 4,Batch : 3000, Los

KeyboardInterrupt: ignored

pretrained 넣기


In [21]:
PATH = '/content/drive/MyDrive/share/trained_model/vgg13_62.95%.pth'



In [22]:
torch.save(net.state_dict(), PATH)

In [16]:
classes = ('plane','automobile','bird','cat','deer',
           'dog','frog','horse','ship','truck')

In [17]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

with torch.no_grad():
  for data in test_loader:
    images, labels = data[0].to(device), data[1].to(device)
    outputs= net(images)
    _, predicted = torch.max(outputs.data,1)
    c= (predicted == labels).squeeze()
    for i in range(4):
      label= labels[i]
      class_correct[label]+= c[i].item()
      class_total[label]+= 1

for i in range(10):
  print("Accuracy of {} : {} %".format(classes[i],100* class_correct[i]/class_total[i]))

Accuracy of plane : 85.71428571428571 %
Accuracy of automobile : 79.16666666666667 %
Accuracy of bird : 33.333333333333336 %
Accuracy of cat : 25.0 %
Accuracy of deer : 75.0 %
Accuracy of dog : 41.666666666666664 %
Accuracy of frog : 45.833333333333336 %
Accuracy of horse : 54.166666666666664 %
Accuracy of ship : 78.57142857142857 %
Accuracy of truck : 54.166666666666664 %


In [18]:
correct = 0
total = 0
with torch.no_grad():
  for data in train_loader:
    images, labels = data[0].to(device), data[1].to(device)
    outputs= net(images)
    _, predicted = torch.max(outputs.data,1)
    total +=labels.size(0)
    correct += (predicted == labels).sum().item()

print(100* correct/total)

97.45


In [20]:
correct = 0
total = 0
with torch.no_grad():
  for data in vaild_loader:
    images, labels = data[0].to(device), data[1].to(device)
    outputs= net(images)
    _, predicted = torch.max(outputs.data,1)
    total +=labels.size(0)
    correct += (predicted == labels).sum().item()

print(100* correct/total)

80.19


In [19]:
correct = 0
total = 0
with torch.no_grad():
  for data in test_loader:
    images, labels = data[0].to(device), data[1].to(device)
    outputs= net(images)
    _, predicted = torch.max(outputs.data,1)
    total +=labels.size(0)
    correct += (predicted == labels).sum().item()

print(100* correct/total)

62.95
